In [1]:
# 環境構築

!apt-get update
!apt-get install -y libsndfile1-dev
!pip install scipy
!pip install scikit-learn
!pip install librosa

/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/allegro/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/a

In [2]:
# clips 作成

SAMPLING_RATE = 44100 # 変更不可

clip_size = 81920 # 楽曲を再構築するパーツ1つあたりの大きさ
step_size = 20480 # clip をずらすときの大きさ
window_size = 10240 # CQT_CHROMA を取得するのに使用するサンプル数
hop_size = 640 # window をずらすときの大きさ

from scipy.io.wavfile import read, write
import glob
import os
import numpy as np
import librosa
import sys

if os.path.isfile("../data/out/clips.wav"):
    print("../data/out/clips.wav exists.")
    files = ["../data/out/clips.wav"]
else:
    # clips.npy をもとにデータを作成する 
    if os.path.isfile("../data/arrays/clips.npy"):
        print("loading ../data/arrays/clips.npy ...")
        clips = np.load("../data/arrays/clips.npy")
        print("creating ../data/out/clips.wav ...")
        write("../data/out/clips.wav", SAMPLING_RATE, clips.reshape((clips.shape[0] * clips.shape[1], )))
        files = ["../data/out/clips.wav"]
        
    # clips,npy がないとき /data/wav44100 内の WAV ファイルを参照する
    else:
        files = glob.glob("../data/wav44100/*.wav")
        #files = ["../data/wav44100/3DEmbodimentFromLines.wav"] # デバッグ用 
        
if len(files) == 0:
    print("WAV ファイルが見つかりませんでした。")
    sys.exit(1)
        
clips_filename = "../data/arrays/c" + str(clip_size) + "_s" + str(step_size) + "_f32_clips"

if os.path.isfile(clips_filename + ".npy"):
    print("loading " + clips_filename + ".npy ...")
    clips = np.load(clips_filename + ".npy")
else:
    print("creating " + clips_filename + ".npy ...")
    raw_data_list = [librosa.load(file, sr=SAMPLING_RATE)[0] for file in files] # 左の音だけ使う

    clips = np.zeros((0, clip_size), dtype=np.float32)
    for raw_data in raw_data_list:
        tmp = [raw_data[i:i + clip_size] for i in range(0, len(raw_data) - clip_size, step_size)]
        clips = np.vstack((clips, np.array(tmp)))
    np.save(clips_filename, clips)

print("The clip array has " + str(clips.shape[0]) + " clips.")

../data/out/clips.wav exists.
loading ../data/arrays/c81920_s20480_f32_clips.npy ...
The clip array has 57148 clips.


In [3]:
# cqt 作成

n_bins = 84

cqt_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqts"

if os.path.isfile(cqt_filename + ".npy"):
    print("loading " + cqt_filename + ".npy ...")
    cqts = np.load(cqt_filename + ".npy")
else:
    print("creating " + cqt_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("cqt progress: clip " + str(num + 1) + " / " + str(len(clips)))
        
        tmp_cqt = librosa.cqt(clip, sr=SAMPLING_RATE, hop_length=hop_size, n_bins=n_bins)
        tmp_cqt = tmp_cqt.reshape((1, tmp_cqt.shape[0], tmp_cqt.shape[1]))
        
        if num == 0:
            cqts = tmp_cqt
        else:
            cqts = np.vstack((cqts, tmp_cqt))
            
        num += 1

    np.save(cqt_filename, cqts)
    
print("The cqt array has " + str(cqts.shape[0]) + " cqts.")
print("cqts.shape: " + str(cqts.shape))
print("Type(cqts[0][0][0]): " + str(type(cqts[0][0][0])))
print("np.max(cqts[0][0]): " + str(np.max(cqts[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips_w10240_h640_cqts.npy ...
The cqt array has 57148 cqts.
cqts.shape: (57148, 84, 129)
Type(cqts[0][0][0]): <class 'numpy.complex64'>
np.max(cqts[0][0]): (0.021538047+0.007131239j)


In [4]:
# cqt_choroma 作成

def Preprocessing(array):
    array = np.abs(array)
    array = np.log(array + 1)
    array = array / np.log(np.finfo(np.float32).max)
    array = array.T
    return array

cqt_chroma_filename = clips_filename + "_w" + str(window_size) + "_h" + str(hop_size) + "_cqt_chromas"

if os.path.isfile(cqt_chroma_filename + ".npy"):
    print("loading " + cqt_chroma_filename + ".npy ...")
    cqt_chromas= np.load(cqt_chroma_filename + ".npy")
else:
    print("creating " + cqt_chroma_filename + ".npy ...")
    num = 0
    for cqt in cqts:
        print("cqt_chroma progress: clip " + str(num + 1) + " / " + str(len(cqts)))
        
        tmp_cqt_chroma = librosa.feature.chroma_cqt(C=cqt, sr=SAMPLING_RATE)
        tmp_cqt_chroma = Preprocessing(tmp_cqt_chroma)
        tmp_cqt_chroma = tmp_cqt_chroma.reshape((1, tmp_cqt_chroma.shape[0], tmp_cqt_chroma.shape[1]))
        
        if num == 0:
            cqt_chromas = tmp_cqt_chroma
        else:
            cqt_chromas = np.vstack((cqt_chromas, tmp_cqt_chroma))
            
        num += 1

    np.save(cqt_chroma_filename, cqt_chromas)
    
print("The cqt_chroma array has " + str(cqt_chromas.shape[0]) + " cqt_chromas.")
print("cqt_chromas.shape: " + str(cqt_chromas.shape)) # clip 番号、window 番号、 chroma 番号 になる
print("Type(cqt_chromas[0][0][0]): " + str(type(cqt_chromas[0][0][0])))
print("np.max(cqt_chromas[0][0]): " + str(np.max(cqt_chromas[0][0])))

loading ../data/arrays/c81920_s20480_f32_clips_w10240_h640_cqt_chromas.npy ...
The cqt_chroma array has 57148 cqt_chromas.
cqt_chromas.shape: (57148, 129, 12)
Type(cqt_chromas[0][0][0]): <class 'numpy.float32'>
np.max(cqt_chromas[0][0]): 0.0078125


In [5]:
# max_norm 作成

num_divide = 4

max_norm_filename = clips_filename + "_d" + str(num_divide) + "_max_norms"

if os.path.isfile(max_norm_filename + ".npy"):
    print("loading " + max_norm_filename + ".npy ...")
    max_norms= np.load(max_norm_filename + ".npy")
else:
    print("creating " + max_norm_filename + ".npy ...")
    num = 0
    for clip in clips:
        print("max_norm progress: clip " + str(num + 1) + " / " + str(len(clips)))
        max_norm = [np.max(clip[int((i / num_divide) * len(clip)):int(((i + 1) / num_divide) * len(clip))] ** 2) for i in range(num_divide)]
    
        if num == 0:
            max_norms = np.array(max_norm)
        else:
            max_norms = np.vstack((max_norms, np.array(max_norm)))
        num += 1
        
    np.save(max_norm_filename, max_norms)
    
print("The max_norm array has " + str(max_norms.shape[0]) + " max_norms.")
print("max_norms.shape: " + str(max_norms.shape))
print("Type(max_norms[0][0]): " + str(type(max_norms[0][0])))
print("np.max(max_norms[0]): " + str(np.max(max_norms[0])))

loading ../data/arrays/c81920_s20480_f32_clips_d4_max_norms.npy ...
The max_norm array has 57148 max_norms.
max_norms.shape: (57148, 4)
Type(max_norms[0][0]): <class 'numpy.float32'>
np.max(max_norms[0]): 0.0179649


In [6]:
# 学習データとテストデータに分ける

def normalize(array):
    if (np.sum(array) == 0):
        return array
    else:
        return array / np.sum(array)
    
cqt_chroma_sum_threshold = 0.01 # 次の window の sum が閾値に満たないときに除外します
test_data_rate = 0.1
weight = 0.01 # max_norm の重み

window_num_per_clip = cqt_chromas.shape[1]

cqt_chromas = np.array([np.hstack((cqt_chromas[i], np.repeat(np.array([max_norms[i]]), cqt_chromas.shape[1], axis=0) * weight)) for i in range(len(max_norms))])
cqt_chromas = np.concatenate([cqt_chromas[:-1, :, :], cqt_chromas[1:, 0, :].reshape(cqt_chromas.shape[0] - 1, 1, cqt_chromas.shape[2])], 1)

p = np.random.permutation(len(cqt_chromas))
cqt_chromas = cqt_chromas[p]
clips = clips[p]

x = np.delete(cqt_chromas, np.where(np.sum(cqt_chromas[:,-1,:], axis=1) < cqt_chroma_sum_threshold)[0], axis=0)

x_test = x[:int(x.shape[0] * test_data_rate), :window_num_per_clip, :].reshape(int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)
x_train = x[int(x.shape[0] * test_data_rate):, :window_num_per_clip, :].reshape(x.shape[0] - int(x.shape[0] * test_data_rate), window_num_per_clip, x.shape[2], 1)
y_tests =  np.array([x[:int(x.shape[0] * test_data_rate), window_num_per_clip, i] for i in range(x.shape[2])])
y_trains = np.array([x[int(x.shape[0] * test_data_rate):, window_num_per_clip, i] for i in range(x.shape[2])])

print("x_train.shape: " + str(x_train.shape))
print("x_test.shape: " + str(x_test.shape))
print("y_trains.shape: " + str(y_trains.shape))
print("y_tests.shape: " + str(y_tests.shape))

x_train.shape: (51237, 129, 16, 1)
x_test.shape: (5692, 129, 16, 1)
y_trains.shape: (16, 51237)
y_tests.shape: (16, 5692)


In [7]:
def plot_history(history):
    # 精度の履歴をプロット
    plt.plot(history.history['accuracy'],"o-",label="accuracy")
    plt.plot(history.history['val_accuracy'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()
 
    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.show()

In [8]:
# モデル作成

import keras

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, Input, MaxPooling2D
from keras.optimizers import Adam

learning_rate = 0.001

models = []
for i in range(x.shape[2]):
    model = keras.models.Sequential()
    model.add(Conv2D(32, (3, 1), activation="relu", input_shape=(window_num_per_clip, cqt_chromas.shape[2], 1)))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.5))
    model.add(Conv2D(64, (3, 1), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.5))
    model.add(Conv2D(128, (3, 1), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.5))
    model.add(Conv2D(256, (3, 1), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 1)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(units=1))
    model.add(Activation('linear'))
    model.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=learning_rate),metrics=['accuracy'])

    model.build()
    model.summary()
    
    models.append(model)

2023-03-11 17:48:56.228883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 17:48:56.558961: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-11 17:48:58.829416: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/allegro/miniconda3/lib/
2023-03-11 17:48:58.833340: W tensorflow/compiler/xla/st

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 16, 32)       128       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 16, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 16, 64)        6208      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 30, 16, 64)        0

2023-03-11 17:49:00.538878: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 17:49:00.538916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-11 17:49:00.541042: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 17:49:00.549540: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 127, 16, 32)       128       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 63, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 63, 16, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 16, 64)        6208      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 16, 64)       0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 30, 16, 64)       

 conv2d_16 (Conv2D)          (None, 127, 16, 32)       128       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 63, 16, 32)       0         
 g2D)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 63, 16, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 61, 16, 64)        6208      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 30, 16, 64)       0         
 g2D)                                                            
                                                                 
 dropout_17 (Dropout)        (None, 30, 16, 64)        0         
                                                                 
 conv2d_18 (Conv2D)          (None, 28, 16, 128)       24704     
          

 g2D)                                                            
                                                                 
 dropout_28 (Dropout)        (None, 63, 16, 32)        0         
                                                                 
 conv2d_29 (Conv2D)          (None, 61, 16, 64)        6208      
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 30, 16, 64)       0         
 g2D)                                                            
                                                                 
 dropout_29 (Dropout)        (None, 30, 16, 64)        0         
                                                                 
 conv2d_30 (Conv2D)          (None, 28, 16, 128)       24704     
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 14, 16, 128)      0         
 g2D)                                                            
          

                                                                 
 conv2d_41 (Conv2D)          (None, 61, 16, 64)        6208      
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 30, 16, 64)       0         
 g2D)                                                            
                                                                 
 dropout_41 (Dropout)        (None, 30, 16, 64)        0         
                                                                 
 conv2d_42 (Conv2D)          (None, 28, 16, 128)       24704     
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 14, 16, 128)      0         
 g2D)                                                            
                                                                 
 dropout_42 (Dropout)        (None, 14, 16, 128)       0         
                                                                 
 conv2d_43

 max_pooling2d_53 (MaxPoolin  (None, 30, 16, 64)       0         
 g2D)                                                            
                                                                 
 dropout_53 (Dropout)        (None, 30, 16, 64)        0         
                                                                 
 conv2d_54 (Conv2D)          (None, 28, 16, 128)       24704     
                                                                 
 max_pooling2d_54 (MaxPoolin  (None, 14, 16, 128)      0         
 g2D)                                                            
                                                                 
 dropout_54 (Dropout)        (None, 14, 16, 128)       0         
                                                                 
 conv2d_55 (Conv2D)          (None, 12, 16, 256)       98560     
                                                                 
 max_pooling2d_55 (MaxPoolin  (None, 6, 16, 256)       0         
 g2D)     

In [9]:
# 学習
batch_size = 64
epochs = 100

for i in range(x.shape[2]):
    model_name = "independent_batch" + str(batch_size) + "_e" + str(epochs) + "_" + str(i)

    history = model.fit(x_train, y_trains[i],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_tests[i]))

    model.save("../data/models/" + model_name)

Epoch 1/100


2023-03-11 17:49:10.355779: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_15/dropout_60/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-03-11 17:49:13.117195: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-11 17:49:16.962136: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-11 17:49:17.000536: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-11 17:49:17.000619: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:85] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-11 17:49:17.037345: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-11 17:49

801/801 [==============================] - 22s 13ms/step - loss: 9.7063e-06 - accuracy: 0.5016 - val_loss: 8.7361e-06 - val_accuracy: 0.4972
Epoch 2/100
801/801 [==============================] - 10s 12ms/step - loss: 8.6695e-06 - accuracy: 0.5016 - val_loss: 8.9393e-06 - val_accuracy: 0.4972
Epoch 3/100
801/801 [==============================] - 10s 12ms/step - loss: 8.6814e-06 - accuracy: 0.5016 - val_loss: 8.7021e-06 - val_accuracy: 0.4972
Epoch 4/100
801/801 [==============================] - 10s 12ms/step - loss: 8.7064e-06 - accuracy: 0.5016 - val_loss: 8.6795e-06 - val_accuracy: 0.4972
Epoch 5/100
801/801 [==============================] - 10s 12ms/step - loss: 8.6897e-06 - accuracy: 0.5016 - val_loss: 8.7134e-06 - val_accuracy: 0.4972
Epoch 6/100
801/801 [==============================] - 10s 12ms/step - loss: 8.7070e-06 - accuracy: 0.5016 - val_loss: 8.7259e-06 - val_accuracy: 0.4972
Epoch 7/100
801/801 [==============================] - 10s 12ms/step - loss: 8.6824e-06 - accu

801/801 [==============================] - 10s 12ms/step - loss: 8.7239e-06 - accuracy: 0.5016 - val_loss: 9.7502e-06 - val_accuracy: 0.4972
Epoch 55/100
801/801 [==============================] - 10s 12ms/step - loss: 8.7105e-06 - accuracy: 0.5016 - val_loss: 8.7042e-06 - val_accuracy: 0.4972
Epoch 56/100
801/801 [==============================] - 10s 12ms/step - loss: 8.7545e-06 - accuracy: 0.5016 - val_loss: 8.8072e-06 - val_accuracy: 0.4972
Epoch 57/100
801/801 [==============================] - 10s 12ms/step - loss: 8.6947e-06 - accuracy: 0.5016 - val_loss: 8.6876e-06 - val_accuracy: 0.4972
Epoch 58/100
801/801 [==============================] - 10s 12ms/step - loss: 8.7019e-06 - accuracy: 0.5016 - val_loss: 8.7437e-06 - val_accuracy: 0.4972
Epoch 59/100
801/801 [==============================] - 10s 12ms/step - loss: 8.6912e-06 - accuracy: 0.5016 - val_loss: 8.7276e-06 - val_accuracy: 0.4972
Epoch 60/100
801/801 [==============================] - 10s 12ms/step - loss: 8.7256e-06 

INFO:tensorflow:Assets written to: ../data/models/independent_batch64_e100_0/assets


...

801/801 [==============================] - 10s 12ms/step - loss: 7.2048e-06 - accuracy: 0.4992 - val_loss: 7.1666e-06 - val_accuracy: 0.5019
Epoch 55/100
801/801 [==============================] - 10s 12ms/step - loss: 7.1970e-06 - accuracy: 0.4992 - val_loss: 7.0186e-06 - val_accuracy: 0.5019
Epoch 56/100
801/801 [==============================] - 10s 12ms/step - loss: 7.2174e-06 - accuracy: 0.4992 - val_loss: 7.2114e-06 - val_accuracy: 0.5019
Epoch 57/100
801/801 [==============================] - 10s 12ms/step - loss: 7.1834e-06 - accuracy: 0.4992 - val_loss: 7.0194e-06 - val_accuracy: 0.5019
Epoch 58/100
801/801 [==============================] - 10s 12ms/step - loss: 7.1970e-06 - accuracy: 0.4992 - val_loss: 7.1601e-06 - val_accuracy: 0.5019
Epoch 59/100
801/801 [==============================] - 10s 12ms/step - loss: 7.2160e-06 - accuracy: 0.4992 - val_loss: 7.2095e-06 - val_accuracy: 0.5019
Epoch 60/100
801/801 [==============================] - 10s 12ms/step - loss: 7.1624e-06 

INFO:tensorflow:Assets written to: ../data/models/independent_batch64_e100_4/assets


INFO:tensorflow:Assets written to: ../data/models/independent_batch64_e100_4/assets


Epoch 1/100
801/801 [==============================] - 10s 12ms/step - loss: 7.0085e-06 - accuracy: 0.5024 - val_loss: 6.9842e-06 - val_accuracy: 0.4858
Epoch 2/100
801/801 [==============================] - 10s 12ms/step - loss: 6.9811e-06 - accuracy: 0.5024 - val_loss: 6.8585e-06 - val_accuracy: 0.4858
Epoch 3/100
801/801 [==============================] - 10s 12ms/step - loss: 7.0020e-06 - accuracy: 0.5024 - val_loss: 6.9289e-06 - val_accuracy: 0.4858
Epoch 4/100
801/801 [==============================] - 11s 14ms/step - loss: 6.9923e-06 - accuracy: 0.5024 - val_loss: 6.8630e-06 - val_accuracy: 0.4858
Epoch 5/100
801/801 [==============================] - 10s 13ms/step - loss: 7.0008e-06 - accuracy: 0.5024 - val_loss: 7.1504e-06 - val_accuracy: 0.4858
Epoch 6/100
801/801 [==============================] - 10s 12ms/step - loss: 6.9909e-06 - accuracy: 0.5024 - val_loss: 7.2029e-06 - val_accuracy: 0.4858
Epoch 7/100
801/801 [==============================] - 13s 17ms/step - loss: 7.013

KeyboardInterrupt: 